# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [30]:
import pandas as pd
import numpy as np
import requests
import json
import matplotlib.pyplot as plt

from api_keys import weather_api_key
from citipy import citipy

In [31]:

#Casts a dataframe with two very long sets of random numbers formatted for use as random lat/long coordinates
cities_df = pd.DataFrame({"long":(np.random.uniform(-1.8,1.8,1200)*100), "lat":(np.random.uniform(-0.9,0.9,1200)*100)})

####
#### API SAVER
####
cities_df = cities_df.truncate(after = 9)


cities_df.head()

#city = citipy.nearest_city(22.99,120.21)
#city.country_code

long        lat
0  19.518144 -79.649273
1  70.849504  69.942734
2  46.448848  27.371486
3 -33.952333  49.769213
4  18.283074  48.027046

## Generate Cities List

In [32]:
cities_name = []
countrycode = []
for i in range(0,len(cities_df['lat'])):
    
    #API saver
    if i>10:
        break
    
    
    lat = cities_df.iloc[i]['lat']
    long = cities_df.iloc[i]['long']
    city = citipy.nearest_city(lat,long)
    
    name = city.city_name
    cities_name.append(name)
    country = city.country_code
    countrycode.append(country)
    
cities_df['City'] = cities_name
cities_df['Country'] = countrycode
cities_df.head(10)

long        lat        City Country
0   19.518144 -79.649273  bredasdorp      za
1   70.849504  69.942734    yar-sale      ru
2   46.448848  27.371486        doha      kw
3  -33.952333  49.769213       lagoa      pt
4   18.283074  48.027046      surany      sk
5 -143.290504  25.703696        hilo      us
6 -109.153852 -26.757531     rikitea      pf
7   -5.450462  19.346518    araouane      ml
8  115.519363  86.368716   saskylakh      ru
9    2.553293  -6.322089      omboue      ga

In [33]:
#drop duplicate cities in cities_df
cities_copy = cities_df.copy()
new_cities = cities_copy.drop_duplicates("City", keep = 'first')

#reset cities_df's index and truncate at 499 to provide sample size of 500 random cities.
new_cities = new_cities.reset_index(drop = True)
new_cities = new_cities.truncate( after=499)
new_cities

long        lat        City Country
0   19.518144 -79.649273  bredasdorp      za
1   70.849504  69.942734    yar-sale      ru
2   46.448848  27.371486        doha      kw
3  -33.952333  49.769213       lagoa      pt
4   18.283074  48.027046      surany      sk
5 -143.290504  25.703696        hilo      us
6 -109.153852 -26.757531     rikitea      pf
7   -5.450462  19.346518    araouane      ml
8  115.519363  86.368716   saskylakh      ru
9    2.553293  -6.322089      omboue      ga

In [34]:
api_key = weather_api_key


#Establish columns for data to be used in scatter plots
new_cities["Temp"]=""
new_cities["Humidity"]=""
new_cities["Cloudiness"]=""
new_cities["Wind (mph)"]=""
new_cities.head(10)

long        lat        City Country Temp Humidity Cloudiness  \
0   19.518144 -79.649273  bredasdorp      za                            
1   70.849504  69.942734    yar-sale      ru                            
2   46.448848  27.371486        doha      kw                            
3  -33.952333  49.769213       lagoa      pt                            
4   18.283074  48.027046      surany      sk                            
5 -143.290504  25.703696        hilo      us                            
6 -109.153852 -26.757531     rikitea      pf                            
7   -5.450462  19.346518    araouane      ml                            
8  115.519363  86.368716   saskylakh      ru                            
9    2.553293  -6.322089      omboue      ga                            

  Wind (mph)  
0             
1             
2             
3             
4             
5             
6             
7             
8             
9

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [35]:
#Use iterrows() to loop through the new_cities sample size and perform API calls for each 

In [36]:
#Test API block status

target_url=f"http://api.openweathermap.org/data/2.5/weather?q={new_cities.City[4]}&appid={weather_api_key}"
weather_json = requests.get(target_url).json()
print(weather_json)
#print(weather_api_key)
#print("       ")
#print("       ")
#print(weather_json["main"]["temp"])


#temp_list.append(weather_json["main"]["temp"])
#print(temp_list)

{'coord': {'lon': 18.18, 'lat': 48.09}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03n'}], 'base': 'stations', 'main': {'temp': 293.15, 'feels_like': 293.14, 'temp_min': 292.59, 'temp_max': 293.71, 'pressure': 1019, 'humidity': 77}, 'visibility': 10000, 'wind': {'speed': 2.77, 'deg': 20}, 'clouds': {'all': 35}, 'dt': 1599773109, 'sys': {'type': 3, 'id': 2003682, 'country': 'SK', 'sunrise': 1599711475, 'sunset': 1599757845}, 'timezone': 7200, 'id': 3057339, 'name': 'Šurany', 'cod': 200}


In [37]:
temp_list = []
humid_list = []
cloud_list = []
wind_list = []

for index, row in new_cities.iterrows():
    
    
    #API saver
    if index>10:
        break
    
    target_url=f"http://api.openweathermap.org/data/2.5/weather?q={row[2]}&appid={weather_api_key}"
    weather_json = requests.get(target_url).json()
    print(f"Processing Record {index} | {row[2]}")
    
    try:
        #new_cities.set_value(index, "Temp", weather_json["main"]["temp"])
        temp_list.append(weather_json["main"]["temp"])
        humid_list.append(weather_json["main"]["humidity"])
        cloud_list.append(weather_json["clouds"]["all"])
        wind_list.append(weather_json["wind"]["speed"])
        
        
        #add in list appends here for Humidity, Cloudiness, and Wind (mph)
        
        
        
        
        #new_cities.set_value(index, "Humidity", weather_json["main"]["humidity"])
        #new_cities.set_value(index, "Cloudiness", weather_json["clouds"]["all"])
        #new_cities.set_value(index, "Wind (mph)", weather_json["wind"]["speed"])
        
    except:
        temp_list.append("")
        humid_list.append("")
        cloud_list.append("")
        wind_list.append("")
        print("City not found. Skipping...")

Processing Record 0 | bredasdorp
Processing Record 1 | yar-sale
Processing Record 2 | doha
Processing Record 3 | lagoa
Processing Record 4 | surany
Processing Record 5 | hilo
Processing Record 6 | rikitea
Processing Record 7 | araouane
Processing Record 8 | saskylakh
Processing Record 9 | omboue


In [39]:
new_cities["Temp"] = temp_list
new_cities["Humidity"] = humid_list
new_cities["Cloudiness"] = cloud_list
new_cities["Wind (mph)"] = wind_list
###
### ADD A DROPNA STATEMENT DOWN BELOW TO REMOVE ALL MISSING OPTIONS
### OR DELETE BASED ON TEMP BEING "NAN" OR "" OR WHATEVER
### JUST MAKE SURE YOU DO IT
###

#print(len(temp_list))
 

def kelv_to_fahr(temp_kelv):
    """Convert Kelvin to Fahrenheit
    
    Return Fahrenheit conversion of input"""
    
    temp_fahr = (temp_kelv)* 9/5 - 459.67
    return temp_fahr


#converts the strings to numbers so they can be formatted into fahrenheit
#new_cities.dropna() to get rid of the empty variables where data was lacking
#or do a reverse loc statement, removing all that has this characteristic - "NaN" or "" 
#before numericization

#To remove all rows where column 'score' is < 50:
#df = df.drop(df[df.score < 50].index)
new_cities = new_cities.drop(new_cities[new_cities.Temp == ""].index)

new_cities = new_cities.reset_index(drop = True)
new_cities["Temp"] = pd.to_numeric(new_cities["Temp"])
new_cities["Temp"] = kelv_to_fahr(new_cities["Temp"])



new_cities.head(10)

C:\Users\riggi\anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


long        lat        City Country     Temp  Humidity  Cloudiness  \
0   19.518144 -79.649273  bredasdorp      za   59.000        87         100   
1   70.849504  69.942734    yar-sale      ru   56.930        76          36   
2   46.448848  27.371486        doha      kw   90.140        79           0   
3  -33.952333  49.769213       lagoa      pt   72.284        83           0   
4   18.283074  48.027046      surany      sk   68.000        77          35   
5 -143.290504  25.703696        hilo      us   76.874        78          90   
6 -109.153852 -26.757531     rikitea      pf   73.328        84         100   
7   -5.450462  19.346518    araouane      ml  102.560        14           0   
8  115.519363  86.368716   saskylakh      ru   45.032        85          95   
9    2.553293  -6.322089      omboue      ga   75.308        82           7   

   Wind (mph)  
0        5.70  
1        8.64  
2        1.00  
3        2.90  
4        2.77  
5        3.60  
6        7.68  
7        3.05  
8        2.16  
9        3.72

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [40]:
#export dataframe as csv
new_cities.head()

long        lat        City Country    Temp  Humidity  Cloudiness  \
0  19.518144 -79.649273  bredasdorp      za  59.000        87         100   
1  70.849504  69.942734    yar-sale      ru  56.930        76          36   
2  46.448848  27.371486        doha      kw  90.140        79           0   
3 -33.952333  49.769213       lagoa      pt  72.284        83           0   
4  18.283074  48.027046      surany      sk  68.000        77          35   

   Wind (mph)  
0        5.70  
1        8.64  
2        1.00  
3        2.90  
4        2.77

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [41]:
#To remove all rows where column 'score' is < 50:
#df = df.drop(df[df.score < 50].index)

new_cities = new_cities.drop(new_cities[new_cities.Humidity > 100].index)
new_cities.head()

long        lat        City Country    Temp  Humidity  Cloudiness  \
0  19.518144 -79.649273  bredasdorp      za  59.000        87         100   
1  70.849504  69.942734    yar-sale      ru  56.930        76          36   
2  46.448848  27.371486        doha      kw  90.140        79           0   
3 -33.952333  49.769213       lagoa      pt  72.284        83           0   
4  18.283074  48.027046      surany      sk  68.000        77          35   

   Wind (mph)  
0        5.70  
1        8.64  
2        1.00  
3        2.90  
4        2.77

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression